In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os

# Set various TF options
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

import tensorflow as tf
import keras.api as keras

tf.debugging.set_log_device_placement(False)

print("TF version:", tf.__version__)
print("TF device:", tf.config.list_physical_devices('GPU')) # or 'CPU'

TF version: 2.17.1
TF device: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Approach 1

## Extracting pixels from images

In [2]:
from common import CV_DATA_DIR
from pathlib import Path

# Pre-processed input images
images = []
# Classes of input images
classes = []
# Shape of input image (height x width = NN inputs)
height, width = 64, 64

# Filter out regular files
files = [
    file for file in sorted(Path(CV_DATA_DIR, 'homer_bart_1').iterdir())
    if file.is_file()
]

# Pre-process images
for file in files:
    try:
        image = cv.imread(str(file))

        # Reshape images to have exact size (each pixel will be as an input of NN)
        image = cv.resize(image, (width, height))
        # Convert to grayscale (reduce amount of neurons in input layer of NN, 49152 vs 16384) 
        image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
        # Convert matrix to a vector
        image = image.ravel()

        images.append(image)
        classes.append(0 if file.name.startswith('b') else 1)
    except Exception as e:
        print(e)

# Form images and classes arrays 
Xs = np.asarray(images)
Ys = np.asarray(classes)

(classes, counts) = np.unique(Ys, return_counts=True)
print(f'Bart class images ({classes[0]}): {counts[0]}')
print(f'Homer class images ({classes[1]}): {counts[1]}')

FileNotFoundError: [Errno 2] No such file or directory: '/home/denys/projects/cv-playground/datasets/homer_bart_1'

## Normalizing data

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Normalize pixel values for better result ([0, 255] => [0, 1])
scaler = MinMaxScaler()
Xs = scaler.fit_transform(Xs)

## Train and test set

In [ ]:
from sklearn.model_selection import train_test_split

# Split a bunch of input images to train and test image groups
Xs_train, Xs_test, Ys_train, Ys_test = train_test_split(Xs, Ys, test_size=0.2, random_state=1)

## Building and training NN

In [ ]:
inputs = width * height
output_classes = 2
hidden_neurons = int((inputs + output_classes) / 2)

# Create neural network (NN)
model0 = keras.Sequential()
model0.add(keras.Input(shape=(inputs,)))
model0.add(keras.layers.Dense(units=hidden_neurons, activation='relu'))
model0.add(keras.layers.Dense(units=hidden_neurons, activation='relu'))
model0.add(keras.layers.Dense(units=1, activation='sigmoid'))
model0.summary()

In [ ]:
# Compile NN and fit
model0.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model0.fit(Xs_train, Ys_train, epochs=50, verbose=1)

## Evaluating NN

In [ ]:
from sklearn.metrics import accuracy_score

predictions = model0.predict(Xs_test)
predictions = (predictions > 0.5)  # (0 - Bart, 1 - Homer)
print("Accuracy: {:.2f}%".format(accuracy_score(Ys_test, predictions)))

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(Ys_test, predictions)
sns.heatmap(cm, annot=True);

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(Ys_test, predictions))

## Saving and loading NN

In [ ]:
from common import CV_WORKAREA_DIR

MODEL_FILE = Path(CV_WORKAREA_DIR, 'home_bart1.json')
MODEL_WEIGHTS_FILE = Path(CV_WORKAREA_DIR, 'home_bart1.keras')

model_json = model0.to_json()
with open(MODEL_FILE, 'w') as json_file:
    json_file.write(model_json)

keras.models.save_model(model0, MODEL_WEIGHTS_FILE)

In [ ]:
with open(MODEL_FILE) as json_file:
    json_model = json_file.read()

model1 = keras.models.model_from_json(json_model)
model1.load_weights(MODEL_WEIGHTS_FILE)
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model1.summary()

## Classifying one single image

In [ ]:
test_image = Xs_test[34]
test_image = scaler.inverse_transform(test_image.reshape(1, -1))

In [ ]:
from common import show_image_plot

show_image_plot(test_image.reshape(width, height))

In [ ]:
prediction = model1.predict(test_image)[0][0]
if prediction < 0.5:
    print("Bart")
else:
    print("Homer")

# Approach 2 (feature extraction)

Will be used feature extraction technique based on color.

Homer features:
 * Brown (mouth)
 * Blue (pants)
 * Gray (shoes)

Bart features:
 * Orange (T-shirt)
 * Blue (shorts)
 * Blue (sneakers)
     

In [ ]:
from common import CV_DATA_DIR
from pathlib import Path

# Filter out regular files
files = [
    file for file in sorted(Path(CV_DATA_DIR, 'homer_bart_1').iterdir())
    if file.is_file()
]

In [ ]:
export = 'mount,pants,shoes,tshirt,shorts,sneakers,class\n'
show_images = True
features = []

## Extract features

In [ ]:
# Extract featured from input images (base featured on difference in body parts and cloths color)
for file in files:
    try:
        origin = cv.imread(str(file))
        (H, W) = origin.shape[:2]
    except Exception as e:
        print(e)
        continue
    
    mouth = pants = shoes = 0
    tshirt = shorts = sneakers = 0
    class_name = 0 if file.name.startswith('b') else 1

    image = origin.copy()
    for height in range(0, H):
        for width in range(0, W):
            B, G, R = (
                image.item(height, width, 0),
                image.item(height, width, 1),
                image.item(height, width, 2)
            )
            # Home - brown mouth
            if 70 <= B <= 185 and 140 <= G <= 185 and 175 <= R <= 215:
                image[height, width] = [0, 255, 255]
                mouth += 1
            # Home - blue pants
            if 150 <= B <= 180 and 98 <= G <= 120 and 0 <= R <= 90:
                image[height, width] = [0, 255, 255]
                pants += 1
            # Home - gray shoes
            if height > (H / 2): # Search only in the bottom part of image
                if 25 <= B <= 45 and 25 <= G <= 45 and 25 <= R <= 45:
                    image[height, width] = [0, 255, 255]
                    shoes += 1
            # Bart - orange t-shirt
            if 11 <= B <= 22 and 85 <= G <= 105 and 240 <= R <= 255:
                image[height, width] = [0, 255, 255]
                tshirt += 1
            # Bart - blue shorts
            if 125 <= B <= 170 and 0 <= G <= 12 and 0 <= R <= 20:
                image[height, width] = [0, 255, 255]
                shorts += 1
            # Bart - blue sneakers
            if height > (H / 2): # Search only in the bottom part of image
                if 125 <= B <= 170 and 0 <= G <= 12 and 0 <= R <= 20:
                    image[height, width] = [0, 255, 255]
                    sneakers += 1         
       
    mouth = round((mouth / (H * W)) * 100, 9)
    pants = round((pants / (H * W)) * 100, 9)
    shoes = round((shoes / (H * W)) * 100, 9)
    tshirt = round((tshirt / (H * W)) * 100, 9)
    shorts = round((shorts / (H * W)) * 100, 9)
    sneakers = round((sneakers / (H * W)) * 100, 9)
    
    items = [mouth, pants, shoes, tshirt, shorts, sneakers, class_name]
    features.append(items)
    
    export += (",".join(str(item) for item in items)) + '\n'

In [ ]:
# Save features into CSV file
from common import CV_WORKAREA_DIR
FEATURES_CSV_FILE = Path(CV_WORKAREA_DIR, 'home_bart.csv')
with open(FEATURES_CSV_FILE, 'w') as file:
    file.write(export)

In [ ]:
# Read saved featured from CSV file
dataset = pd.read_csv(FEATURES_CSV_FILE)
Xs = dataset.iloc[:, 0:6].values
Ys = dataset.iloc[:, 6].values

In [ ]:
# Split input dataset into train and test groups
from sklearn.model_selection import train_test_split
Xs_train, Xs_test, Ys_train, Ys_test = train_test_split(Xs, Ys, test_size=0.2, random_state=1)

## Building and training the neural network

In [ ]:
# Input image shape has six column (we need only 6 inputs)
inputs = Xs_train.shape[1]
# Output classes is binary (0 - Bart, 1 - Homer)
output_classes = 2
# Calculates the amount of neurons in hidden layers 
hidden_neurons = int((inputs + output_classes) / 2)

In [ ]:
# Create NN
model2 = keras.Sequential()
model2.add(keras.Input(shape=(inputs,)))
model2.add(keras.layers.Dense(units=hidden_neurons, activation='relu'))
model2.add(keras.layers.Dense(units=hidden_neurons, activation='relu'))
model2.add(keras.layers.Dense(units=hidden_neurons, activation='relu'))
model2.add(keras.layers.Dense(units=1, activation='sigmoid'))
model2.summary()

In [ ]:
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model2.fit(Xs_train, Ys_train, epochs=50, verbose=1)

## Evaluating NN

In [ ]:
plt.plot(history.history['loss']);

In [ ]:
plt.plot(history.history['accuracy']);

In [ ]:
plt.subplot(Xs_train, Ys_train)

predictions = model2.predict(Xs_test)
predictions = (predictions > 0.5)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(Ys_test, predictions)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Ys_test, predictions)
sns.heatmap(cm, annot=True);

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Ys_test, predictions))

## Saving, loading and classifying one single image

In [ ]:
from common import CV_WORKAREA_DIR

MODEL_FILE = Path(CV_WORKAREA_DIR, 'home_bart2.json')
MODEL_WEIGHTS_FILE = Path(CV_WORKAREA_DIR, 'home_bart2.keras')

model_json = model2.to_json()
with open(MODEL_FILE, 'w') as json_file:
    json_file.write(model_json)

keras.models.save_model(model2, MODEL_WEIGHTS_FILE)
with open(MODEL_FILE) as json_file:
    json_model = json_file.read()

In [ ]:
model3 = keras.models.model_from_json(json_model)
model3.load_weights(MODEL_WEIGHTS_FILE)
model3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model3.summary()

In [ ]:
image = Xs_test[0] # Contains only features
print(image)

In [ ]:
image = image.reshape(1, -1)
prediction = model3.predict(image)
if prediction < 0.5:
    print('Bart')
else:
    print('Homer')